In [1]:
# pip install fosforml

In [2]:
# pip show fosforml

In [3]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

/opt/conda/lib/python3.10/site-packages/fosforml-1.2.1.3-py3.10.egg/fosforml/model_manager/model_registry.py:101: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.10/site-packages/fosforml-1.2.1.3-py3.10.egg/fosforml/model_manager/model_registry.py:174: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.10/site-packages/fosforml-1.2.1.3-py3.10.egg/fosforml/model_manager/model_registry.py:101: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.10/site-packages/fosforml-1.2.1.3-py3.10.egg/fosforml/model_manager/model_registry.py:174: SyntaxWarning: "is not" with a literal. Did you mean "!="?
2025-04-01 12:20:05,463 - fdc_connector_python_sdk.connectors.snowflake - INFO - starting the snowpark session creation
2025-04-01 12:20:07,632 - fdc_connector_python_sdk.connectors.snowflake - INFO - time taken to create snowflake session 2.1694142818450928


In [4]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [5]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [ ]:
import pandas as pd
df = pd.read_csv("noteboos/notebooks/HR_Chrun_NB/Notebook_Run/HR_CHURN.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [10]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                  |157                      |3                     |False            |True            |False                    |
|2.0       |7.0           |0.8                   |0.86               |5                  |262                      |6                     |False            |True            |False                 

In [11]:
pipeline = RandomForestRegressor(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [12]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [13]:
from fosforml import register_model

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="snow_12dec_HR_Churn_snow_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_9FEB4029_B86E_4724_B4E6_7016390DCFF1_FDC_SNOW_12DEC_HR_CHURN_SNOW_3_10' registered successfully."

: 